<a href="https://colab.research.google.com/github/omkar806/on-device-experiments/blob/main/Supabase_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install supabase==2.3.0 fastapi==0.104.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00


In [2]:
from fastapi import HTTPException, Body, Header, Request, Depends

In [4]:
!pip install python-dotenv pydantic

In [6]:
!pip install pydantic-settings==2.1.0

In [8]:
from pydantic_settings import BaseSettings
from dotenv import load_dotenv
from supabase import create_client
from functools import lru_cache
import os
from google.colab import userdata
load_dotenv()


# Settings
# class Keys(BaseSettings):
#     SUPABASE_ANON_KEY: str
#     SUPABASE_SERVICE_KEY: str
#     SUPABASE_URL: str
#     Vector_DB_CONNECTION: str
#     VIBE_GUEST_KEY: str
#     SEARCH_PRODUCTS_URL: str
#     FILTER_SEARCH_URL: str

#     class Config:
#         env_file = ".env"


@lru_cache(maxsize=1)
class Supabase_Settings(BaseSettings):

    # keys: BaseSettings = Keys()

    SUPABASE_URL: str = userdata.get("SUPABASE_URL")
    SUPABASE_ANON_KEY: str = userdata.get("SUPABASE_ANON_KEY")
    SUPABASE_SERVICE_KEY: str = userdata.get("SUPABASE_SERVICE_KEY")



@lru_cache(maxsize=1)
class Supabase_Clients:
    values = Supabase_Settings()

    # Depreciated
    # vecs.collection.Collection = lambda *args, **kwargs: CustomCollection(*args, **kwargs)
    # vx = vecs.create_client(values.Vector_DB_CONNECTION)
    # vector_table = vx.get_or_create_collection(name="ProductEmbeddings", dimension=512)

    anon_supabase = create_client(values.SUPABASE_URL, values.SUPABASE_ANON_KEY)
    service_supabase = create_client(values.SUPABASE_URL, values.SUPABASE_SERVICE_KEY)


In [24]:
def AuthUser_Validator(apitoken: str):
    supabase_clients = Supabase_Clients()
    settings = Supabase_Settings()
    supabase = supabase_clients.anon_supabase
    try:
        user = supabase.auth.get_user(apitoken)
        print(user.user.id)
        return str(user.user.id)
    except Exception as e:
        print("Exceptional error")
        if apitoken == settings.SUPABASE_ANON_KEY:
            return "Exceptional error"
            raise HTTPException(
                status_code=401,
                detail=f"invalid Api Token: unable to parse or verify signature, signing method (alg) is unavailable.",
            )

In [25]:
token_1 = "eyJhbGciOiJIUzI1NiIsImtpZCI6IkJQalV4SEMzNUg4NjBDdlgiLCJ0eXAiOiJKV1QifQ.eyJhdWQiOiJhdXRoZW50aWNhdGVkIiwiZXhwIjoxNzE0NDc5MTAyLCJpYXQiOjE3MTQ0NzU1MDIsImlzcyI6Imh0dHBzOi8vZWFlb2t5ZWZzZGZhbXdxcXpma28uc3VwYWJhc2UuY28vYXV0aC92MSIsInN1YiI6IjU2YWU0MGFjLTAxOGMtNGQ5Yi05MzAyLWQyZjI0YmQzYjcxMyIsImVtYWlsIjoieWFzaG1ha2FuLndvcmtAZ21haWwuY29tIiwicGhvbmUiOiIiLCJhcHBfbWV0YWRhdGEiOnsicHJvdmlkZXIiOiJnb29nbGUiLCJwcm92aWRlcnMiOlsiZ29vZ2xlIl19LCJ1c2VyX21ldGFkYXRhIjp7ImF2YXRhcl91cmwiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQ2c4b2NLRWVIYXpuRVhrQXUxZXJEZmtQLTZnclhVekl2N2VRcHlpazROcUVyVkRqZ0dLZ09rPXM5Ni1jIiwiZW1haWwiOiJ5YXNobWFrYW4ud29ya0BnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZnVsbF9uYW1lIjoiWWFzaCBNYWthbiIsImlzcyI6Imh0dHBzOi8vYWNjb3VudHMuZ29vZ2xlLmNvbSIsIm5hbWUiOiJZYXNoIE1ha2FuIiwicGhvbmVfdmVyaWZpZWQiOmZhbHNlLCJwaWN0dXJlIjoiaHR0cHM6Ly9saDMuZ29vZ2xldXNlcmNvbnRlbnQuY29tL2EvQUNnOG9jS0VlSGF6bkVYa0F1MWVyRGZrUC02Z3JYVXpJdjdlUXB5aWs0TnFFclZEamdHS2dPaz1zOTYtYyIsInByb3ZpZGVyX2lkIjoiMTA1MjIzNjI3MjE0MzUxMzQ5ODA2Iiwic3ViIjoiMTA1MjIzNjI3MjE0MzUxMzQ5ODA2In0sInJvbGUiOiJhdXRoZW50aWNhdGVkIiwiYWFsIjoiYWFsMSIsImFtciI6W3sibWV0aG9kIjoib2F1dGgiLCJ0aW1lc3RhbXAiOjE3MTQxMDczODh9XSwic2Vzc2lvbl9pZCI6ImZjN2VjYTExLWVkNjMtNDQwOC1hNTFkLWM1OTM4NDAxZDQ1YiIsImlzX2Fub255bW91cyI6ZmFsc2V9.dBL__fuTuQSYa1d3GQHXo_Th_2upxaPUg_RbSoYNS70"

In [28]:
AuthUser_Validator(token_1)

Exceptional error
